In [15]:
import os
os.chdir("../")
%pwd

'c:\\Users\\RafaelRaiserQSOFT'

In [16]:
import pandas as pd

cols2read=['alpha','delta','u','g','r','i','z','redshift','class']

data = pd.read_csv('artifacts/data_ingestion/data.csv', usecols=cols2read)
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/data.csv'

In [ ]:
data['class'].unique()

array(['GALAXY', 'QSO', 'STAR'], dtype=object)

In [ ]:
mapping = {'GALAXY': 0, 'STAR': 1, 'QSO': 2}

data['class'] = data['class'].replace(mapping)

data.head()

C:\Users\RafaelRaiserQSOFT\AppData\Local\Temp\ipykernel_23252\3795905522.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['class'] = data['class'].replace(mapping)


,alpha,delta,u,g,r,i,z,class,redshift
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,0,0.634794
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,0,0.779136
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,0,0.644195
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,0,0.932346
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,0,0.116123


In [ ]:
data.dtypes

alpha       float64
delta       float64
u           float64
g           float64
r           float64
i           float64
z           float64
class         int64
redshift    float64
dtype: object

In [ ]:
X = data.drop(labels=['class'], axis=1)
y= data['class']

In [ ]:
X.head()

,alpha,delta,u,g,r,i,z,redshift
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123


In [ ]:
y.value_counts()

class
0    59445
1    21594
2    18961
Name: count, dtype: int64

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    target_encoding_path: Path
    TARGET_COLUMN: str

In [ ]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            target_encoding_path=config.target_encoding_path,
            TARGET_COLUMN=schema.name
        )
        return data_transformation_config

In [ ]:
import os
from src import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from src.utils.common import save_bin

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config


    def train_test_spliting(self):

        cols2read = ['alpha','delta','u','g','r','i','z','redshift','class']
        data = pd.read_csv(self.config.data_path, usecols=cols2read)

        logger.info('Data loaded')

        mapping = {'GALAXY': 0, 'STAR': 1, 'QSO': 2}
        data[self.config.TARGET_COLUMN] = data[self.config.TARGET_COLUMN].replace(mapping)
        save_(path = Path(self.config.target_encoding_path), data = mapping)

        logger.info('Target encoding done')

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data, random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [ ]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-02-15 14:33:54,944: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-15 14:33:54,944: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-15 14:33:54,960: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-15 14:33:54,960: INFO: common: created directory at: artifacts]
[2024-02-15 14:33:54,960: INFO: common: created directory at: artifacts/data_transformation]
[2024-02-15 14:33:55,168: INFO: 1747273068: Data loaded]


C:\Users\RafaelRaiserQSOFT\AppData\Local\Temp\ipykernel_23252\1747273068.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[self.config.TARGET_COLUMN] = data[self.config.TARGET_COLUMN].replace(mapping)


EnsureError: Argument path of type <class 'str'> to <function save_pickle at 0x00000207F7F37BA0> does not match annotation type <class 'pathlib.Path'>